# Segmentation with DeepBrain
In this notebook we cover the topic of MRIs segmentation. Although not directly related with the main project's topic, segentation cal help us improve the accuracy of our models, since only the relevant voxels will feed our AE models. We see this as an important part of the pre-process stage.

DeepBrain implements a U-Net architecture to get masks from brain tissues, so it seems perfectly aligned with the goals of our project. In this notebook, we execute deepbrain Extractor over the whole dataset, i.e. all the volumes, in order to get masks for all of them. Every mask can be seen af a volume of voxels indicating wich voxes.

Once we have obtained all the masks, we package them in a zip that can be downloaded. The obtained masks can be then applied to the related volume, getting the relevant brain tissues we are interested in.

Installing Deepbrain is tricky because to get it working we need to downgrade from tensorflow 2.X (default in kaggle) and install tf 1.13.1:

In [ ]:
# Deepbrain requirements:
!pip install tensorflow==1.13.1
!pip install tensorflow-gpu==1.13.1
!pip install deepbrain

In [ ]:
import tensorflow as tf
print(tf.__version__)

Sometimes, kaggle raises an error when loading tf 1.x. To make sure tf1 is properly loaded, a workaround cosistis of restarting the kernel. Once done the tf version should show the correct one.

The nexts piece of code is used to obtaing deepbrain masks for all the volumes:

In [ ]:
import numpy as np
import nibabel as nib
from deepbrain import Extractor
import glob
import os


def get_mask(img_path):
    # Load a nifti as 3d numpy image [H, W, D]
    img = nib.load(img_path).get_fdata()

    ext = Extractor()

    # `prob` will be a 3d numpy image containing probability
    # of being brain tissue for each of the voxels in `img`
    prob = ext.run(img)

    # mask can be obtained as:
    mask = prob > 0.5
   
    # We save the mask to use it later. We preserve the original file name to identify it later when we
    # need to apply it to an MRI.
    np.save(f'{os.path.basename(img_path)}.mask', mask)
    
    # TODO: Perhaps it woukd be a good idea to save also the probabilities matrix
   
    
    
def get_masks():
    # This notebook was executed in kaggle. Kaggle automatically unpacked the uploaded
    # dataset, so the extension to match is .nii rather than .gz
    files = glob.glob('../input/ixi-t1-dataset/*.nii')
    
    # We call get_mask(), a function that uses deepbrain to get the mask of every volume
    # We then will download a zip with all the masks to be used later in the project
    for file in files:
        get_mask(file)        
    
get_masks()    

Once the masks are generated, we are interested to get all of them in a zip so that we can dowload them all to work in a local environment:

In [ ]:
!zip ../../kaggle/working/download.zip ../../kaggle/working/*